In [ ]:
import torch
import torch.nn as nn
import math

In [ ]:
class InputEmbeddings(nn.Module):

  def __init__(self,d_model:int, vocab_size:int):
    super().__init()
    self.d_model = d_model #dimension
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self, x):
    return self.embedding(x) * math.sqrt(self.d_model)


In [ ]:
class PositionalEncoding(nn.Module):
  #seq_len is max sentence length as each word needs a pos enc of size 512(d_model), drpout is to prevent overfitting
  def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.dropout = nn.Dropout(dropout)

    #create a [] of shape seq_len x d_model
    pe=torch.zeros(seq_len, d_model)

     #create a vector of shape (seq_len,1) for position
    position = torch.arange(0,seq_len, d_type = torch.float).unsqueeze(1) #creates tensor([[0],[1],.....[511]]) col vector
    div_term = torch.exp(torch.arange(0,d_model,2).float() * (-math.log(10000.0) / d_model)) #term = e^-(2i(log1000)/d) minus cuz we want the exp to decrease as pos/term

    #apply sin to even positions
    pe[:,0::2] = torch.sin(position*div_term)
    pe[:, 1::2] = torch.cos(position*div_term)

    #for batches of sentences
    pe = pe.pe.unsqueeze(0) #(1, seq_len, d_model)
    self.register_buffer('pe', pe)


  def forward(self, x):
    x = x+(self.pe[:, :x.shape[1], :]).requires_grad_(False)
    return self.dropout(x)



In [ ]:
class LayerNormalization(nn.Module):
  def __init__(self,eps: float=10**-6) -> None:
    super().__init__()
    self.eps = eps
    self.alpha = nn.Parameter(torch.ones(1)) #multiplied
    self.bias= nn.Parameter(torch.zeros(1)) #added

  def forward(self,x):
    mean  = x.mean(dim=-1, keepdim=True) # something like [[batch,seq_len,512]]
    std = x.std(dim=-1, keepdim=True)
    return self.alpha * (x-mean) / (std+self.eps) + self.bias

In [ ]:
class FeedForwardBlock(nn.Module):
  def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
    super().__init__()
    self.linear_1 = nn.Linear(d_model, d_ff) #W1 and b1
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(d_ff, d_model) #W2 and b2

  def forward(self,x):
    return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))



In [ ]:
#multihead attention
class MultiheadAttentionBlock(nn.Module):
  def __init__(self, d_model: int, h: int, dropout: float) -> None:
    super().__init__()
    self.d_model = d_model
    self.h = h
    assert d_model % h ==0, "d_model is not divisible by h"

    self.d_k = d_model // h
    self.W_q = nn.Linear(d_model, d_model)
    self.W_k = nn.Linear(d_model, d_model)
    self.W_v = nn.Linear(d_model, d_model)
    self.W_o = nn.Linear(d_model, d_model)

    self.dropout = nn.Dropout(dropout)

  @staticmethod
  def attention(query, key, value, mask, dropout: nn.Dropout):
    d_k=query.shape[-1]
    #(Batch, h, seq_len, d_k) --> (Batch, h, seq_len, seq_len)
    attention_scores = (query @ key.transpose(-2,-1)) / math.sqrt(d_k)
    if mask is not None:
      attention_scores.masked_fill_(mask == 0,-1e9)
    attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len)

    if dropout is not None:
      attention_scores = dropout(attention_scores)

    return (attention_scores @ value), attention_scores


  def forward(self, q, k, v, mask):
    query = self.W_q(q)
    key = self.W_k(k)
    value = self.W_v(v)

    query = query.view(query.shape[0], query.shape[1],self.h, self.d_k).transpose(1,2)
    key = key.view(query.shape[0], query.shape[1],self.h, self.d_k).transpose(1,2)
    value = value.view(query.shape[0], query.shape[1],self.h, self.d_k).transpose(1,2)

    x, self.attention_scores = MultiheadAttentionBlock.attention(query, key, value, mask, self.dropout)

    #Now concat all heads
    # (Batch, h, seq, d_k) => (Batch, seq, h, d_k) => (Batch, seq, d_model)
    x = x.transpose(1,2).contiguous().view(x.shape[0], -1, self.h*self.d_k)

    return self.W_o(x)


In [ ]:
class ResidualConnection(nn.Module):
  def __init__(self, dropout: float)->None:
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.norm = LayerNormalization()

  def forward(self, x, sublayer):
    return x + self.dropout(sublayer(self.norm(x)))



In [ ]:
class EncoderBlock(nn.Module):
  def __init__(self, self_attention_block:MultiheadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float)->None:
    super().__init__()
    self.self_attention_block = self_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])
#src_mask - to stop padding words from interacting with opther words
  def forward(self, x, src_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
    x = self.residual_connections[1](x, self.feed_forward_block)
    return x

In [ ]:
class Encoder(nn.Module):
  def __init__(self, layers: nn.ModuleList) -> None:
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization()

  def forward(self, x, mask):
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)